In [94]:
import pandas as pd
from gspread_pandas import Spread, Client
import random
import numpy as np
from time import localtime, strftime
import gspread
from gspread_formatting import *

In [95]:
# Authenticate Google Sheets
gc = gspread.oauth()

In [96]:
# Get input and output spreadsheet names, open output spreadsheet
input_spreadsheet_names = list(input("Enter input spreadsheet names (sep by commas): ").split(","))
output_sheet_name = input("Enter name of output spreadsheet: ")
entries_per_sample = int(input("Enter the number of samples to be taken per input sheet: "))
num_samples = int(input("Enter the number of times to randomly select the number of samples: "))

Enter input spreadsheet names (sep by commas): CA - Consolidated Output - 01 to 05
Enter name of output spreadsheet: CA Outputs - Jun 6, 2022 Check
Enter the number of samples to be taken per input sheet: 5
Enter the number of times to randomly select the number of samples: 5


In [97]:
output_spreadsheet_name = Spread(output_sheet_name)
output_gspread = gc.open(output_sheet_name)

stamp_format = cellFormat(
    backgroundColor=color(1, 0.9, 0.9),
    textFormat=textFormat(bold=False, foregroundColor=color(0,0,0)),
    horizontalAlignment='CENTER'
)

def stamp_output(output_sheet_name,url):
    current_time = str(strftime("%Y-%m-%d %H:%M:%S",localtime()))
    hyperlink = '=HYPERLINK("'+url+'","Link to Original")'
    output.update_cells("A1","C1", ["Generated by: spencer@casetext.com",current_time,hyperlink])
    format_cell_range(output_gspread.worksheet(output_sheet_name), 'A1:C1', stamp_format)

header_format = cellFormat(
    textFormat=textFormat(bold=True),
)

def format_header(output_sheet_name):
    format_cell_range(output_gspread.worksheet(output_sheet_name), '2', header_format)

def format_worksheet(output_sheet_name):
    output_gspread.worksheet(output_sheet_name).columns_auto_resize(0,output_gspread.worksheet(output_sheet_name).col_count)

In [98]:
def create_input_sheet_dict(sheet_names):
    inputs_dict = {}
    for sheet_name in sheet_names:
        key_name = sheet_name[0:7]
        sheet_gspread = gc.open(sheet_name)
        spreadsheet_url = "https://docs.google.com/spreadsheets/d/%s" % sheet_gspread.id
        input_data = {"df": Spread(sheet_name).sheet_to_df(),
                     "url": "https://docs.google.com/spreadsheets/d/%s" % sheet_gspread.id,
                     "batch": key_name[6]
                    }
        inputs_dict[key_name] = input_data
    return inputs_dict
    
def create_output(inputs_data_dict, output_name, entries_per_sample, num_samples):
    for sample in range(num_samples):
        print("Batch loading...")
        for sheet,values in inputs_data_dict.items():
            firm_indices = []
            df = pd.DataFrame(values["df"])
            firm_indices = list(set(df.index.values.tolist()))
            while("" in firm_indices):
                firm_indices.remove("")
            if len(firm_indices) > entries_per_sample:
                firm_indices = random.sample(firm_indices, entries_per_sample)
                df = df.loc[firm_indices]
                tab_name = "Sample " + str(sample+1)
                output.df_to_sheet(df,
                        index=True,
                        start='A2',
                        sheet=tab_name,
                        freeze_headers=True,
                        )
            stamp_output(tab_name,values["url"])
            format_header(tab_name)
            format_worksheet(tab_name)
            print(f"Tab {sample+1} uploaded successfully.")
    print(f"All sheets loaded successfully. Outputs: https://docs.google.com/spreadsheets/d/{output_gspread.id}")

In [99]:
clean_input_data = create_input_sheet_dict(input_spreadsheet_names)

In [100]:
create_output(clean_input_data,output_sheet_name,entries_per_sample,num_samples)

Batch loading...
Tab 1 uploaded successfully.
Batch loading...
Tab 2 uploaded successfully.
Batch loading...
Tab 3 uploaded successfully.
Batch loading...
Tab 4 uploaded successfully.
Batch loading...
Tab 5 uploaded successfully.
All sheets loaded successfully. Outputs: https://docs.google.com/spreadsheets/d/1hE6OGgHMFziBuzSNlbK3T6POx1KDyI1EKUsS5WOAXyQ
